In [2]:
import gradio as gr
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import pipeline
import matplotlib.pyplot as plt
import io
import base64

2024-08-26 13:50:01.217835: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-26 13:50:01.476371: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-26 13:50:01.570177: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-26 13:50:01.596747: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-26 13:50:01.805156: I tensorflow/core/platform/cpu_feature_guar

In [3]:
# Initialize sentiment analysis models
vader_analyzer = SentimentIntensityAnalyzer()
transformer_analyzer = pipeline('sentiment-analysis', model="distilbert-base-uncased-finetuned-sst-2-english")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [4]:
# Define sentiment analysis functions
def analyze_sentiment(text):
    # TextBlob
    textblob_polarity = TextBlob(text).sentiment.polarity
    textblob_sentiment = 'Positive' if textblob_polarity > 0 else 'Negative' if textblob_polarity < 0 else 'Neutral'
    
    # VADER
    vader_score = vader_analyzer.polarity_scores(text)['compound']
    vader_sentiment = 'Positive' if vader_score > 0 else 'Negative' if vader_score < 0 else 'Neutral'
    
    # Transformers
    transformer_result = transformer_analyzer(text, truncation=True, max_length=512)
    transformer_sentiment = transformer_result[0]['label']

    # Return both the sentiment results and the plot
    return {
        "TextBlob Sentiment": textblob_sentiment,
        "VADER Sentiment": vader_sentiment,
        "Transformer Sentiment": transformer_sentiment
    }

In [5]:
# Create Gradio interface
iface = gr.Interface(
    fn=analyze_sentiment,
    inputs=gr.Textbox(lines=5, placeholder="Enter text here..."),
    outputs=[gr.JSON()],
    title="Sentiment Analysis App",
    description="Enter text to analyze sentiment using TextBlob, VADER, and Transformers.",
    examples=[["I love this course!"], ["This was a terrible experience."]],
    live=True
)

In [6]:
# Launch the app
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [ ]:
iface.close()